In [1]:
from typing import List, Dict, Set, Any, Optional, Tuple, Literal, Callable
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from tqdm import tqdm
import numpy as np
import torch
from torch import Tensor
from kernels.static_kernels import LinearKernel, RBFKernel
from kernels.sig_trunc import TruncSigKernel

from features.random_fourier import RBF_RandomFourierFeatures
from features.random_sig_fourier import SigTensorisedRandProj
from features.random_warping_series import RandomWarpingSeries
from features.signature import sig, logsig

In [ ]:
# #######################
# ######### RWS #########
# #######################

def RWS_test():
    # Setup and parameters
    N = 3
    N2 = 2
    T = 70
    d = 2
    dtype = torch.float32
    #torch.manual_seed(3)
    X = torch.randn(N, T, d, dtype=dtype).to("cuda").detach()
    Y = torch.randn(N2,T, d, dtype=dtype).to("cuda").detach()
    n_features = 20
    D_min=2
    D_max=5
    sigma=1

    # RWS
    rws = RandomWarpingSeries(n_features, D_min, D_max, sigma, LinearKernel(scale=1/d/T))
    rws.fit(X)
    feat_X = rws.transform(X)
    feat_Y = rws.transform(Y)
    K_rws = 1 + feat_X @ feat_Y.T
    print(feat_X)
    print(feat_X.shape)
    print(K_rws)

RWS_test()

In [ ]:
###############################
#### Lib signatory wrapper ####
###############################

def test_signatory():
    N,T,d = 1, 30, 2
    trunc_level = 3
    X = torch.randn(N, T, d)
    sigs = sig(X, trunc_level) # potential unknown bug. Says imput is not 3D, but it clearly is.
    logsigs = logsig(X, trunc_level)
    print("sigs", sigs.shape, sigs)
    print("logsigs", logsigs.shape, logsigs)

# test_signatory()

In [ ]:
##########################################################
#### Linear TRP-RFSF features  vs  Vanilla Sig kernel ####
##########################################################

def LINEAR_trp_vs_kernel():
    #parameters
    N = 3
    N2 = 2
    T = 20
    d = 2
    trunc_level = 5
    n_features = 100
    dtype = torch.float32
    #torch.manual_seed(3)
    X = torch.randn(N, T, d, dtype=dtype).to("cuda").detach() / np.sqrt(d)
    Y = torch.randn(N2,T, d, dtype=dtype).to("cuda").detach() / np.sqrt(d)

    #exact sig kernel
    sigker = TruncSigKernel(LinearKernel(), normalize=False, trunc_level=trunc_level, geo_order=1, max_batch=50000)
    K = sigker(X,Y)
    print("exact\n", K)

    #trp
    MC_iter = 10000
    res = []
    for i in tqdm(range(MC_iter)):
        trp = SigTensorisedRandProj(trunc_level, n_features, only_last=True, method="linear")
        trp.fit(X)
        feat_X = trp.transform(X)
        feat_Y = trp.transform(Y)
        K_trp = 1 + feat_X @ feat_Y.T
        res.append(K_trp)
    res = torch.stack(res)
    example = res[0]
    mean = res.mean(dim=0)
    print("mean\n", mean)
    print("example\n", example)

LINEAR_trp_vs_kernel()
# exact
#  tensor([[ 4627.0444, -5744.9629],
#         [ 9709.9150, -7802.3965],
#         [-1727.5579,  2563.8979]], device='cuda:0')
# 100%|██████████| 10000/10000 [00:15<00:00, 641.01it/s] # n_features = 500, trunc_level = 5
# mean
#  tensor([[ 4624.3896, -5737.0781],
#         [ 9733.1250, -7796.5479],
#         [-1733.9088,  2575.4778]], device='cuda:0')

In [ ]:
#######################################################
#### Gaussian TRP-RFSF features  vs  SigRBF kernel ####
#######################################################

def GAUSSIAN_trp_vs_kernel():
    #parameters
    N = 3
    N2 = 2
    T = 20
    d = 2
    trunc_level = 5
    n_features = 500
    sigma = 1.0
    dtype = torch.float32
    #torch.manual_seed(3)
    X = torch.randn(N, T, d, dtype=dtype).to("cuda").detach() / np.sqrt(d)
    Y = torch.randn(N2,T, d, dtype=dtype).to("cuda").detach() / np.sqrt(d)

    #exact sig kernel
    sigker = TruncSigKernel(RBFKernel(sigma=sigma), normalize=False, trunc_level=trunc_level, geo_order=1, max_batch=50000)
    K = sigker(X,Y)
    print("exact\n", K)

    #trp
    MC_iter = 10000
    res = []
    for i in tqdm(range(MC_iter)):
        trp = SigTensorisedRandProj(trunc_level, n_features, only_last=True, method="RBF", sigma_rbf=sigma)
        trp.fit(X)
        feat_X = trp.transform(X)
        feat_Y = trp.transform(Y)
        K_trp = 1 + feat_X @ feat_Y.T
        res.append(K_trp)
    res = torch.stack(res)
    example = res[0]
    mean = res.mean(dim=0)
    print("mean\n", mean)
    print("example\n", example)

GAUSSIAN_trp_vs_kernel()
# GAUSSIAN_trp_vs_kernel()
# # exact
# #  tensor([[ -8.7953, -30.1428],
# #         [ -4.1622,  -4.5515],
# #         [-25.4650,  42.8830]], device='cuda:0')
# # 100%|██████████| 10000/10000 [05:55<00:00, 28.10it/s] # n_features=5000, trunc_level=5
# # mean
# #  tensor([[ -9.9236, -30.5771],
# #         [ -4.4291,  -4.5940],
# #         [-25.3243,  43.3625]], device='cuda:0')

In [3]:
############################################################
#### Test RBF_RandomFourierFeatures vs exact RBF kernel ####
############################################################
def rff_vs_exact_RBFKernel():
    N=3
    N2= 2
    d = 10
    sigma=1
    dtype = torch.float64
    # torch.manual_seed(1)
    X = torch.randn(N, d, dtype=dtype).to("cuda") /np.sqrt(d)
    Y = torch.randn(N2, d, dtype=dtype).to("cuda") / np.sqrt(d)

    # Exact RBF kernel
    k = RBFKernel(sigma=sigma)
    K = k(X, Y)

    # Approximate RBF kernel using RBF_RandomFourierFeatures
    N_MC = 10000
    res = []
    for i in range(N_MC):
        RFF = RBF_RandomFourierFeatures(n_features=1000,
                                        sigma=sigma,
                                        method="cos(x)sin(x)",
                                        # method = "cos(x + b)",
                                        )
        RFF.fit(X)
        feat_X = RFF.transform(X)
        feat_Y = RFF.transform(Y)
        K_rff = feat_X @ feat_Y.T
        res.append(K_rff)
    K_rff = torch.mean(torch.stack(res), dim=0)

    print("K\n",K)
    print("K_rff\n",K_rff)
    print("diff\n", K-K_rff)
    print("diffmean\n", torch.mean(abs(K-K_rff)))
    # the RFF approach cant reproduce results smaller than 1e-5 for some reason
    
rff_vs_exact_RBFKernel()

K
 tensor([[0.4836, 0.5957],
        [0.4566, 0.4028],
        [0.2332, 0.2142]], device='cuda:0', dtype=torch.float64)
K_rff
 tensor([[0.4835, 0.5957],
        [0.4568, 0.4027],
        [0.2331, 0.2142]], device='cuda:0', dtype=torch.float64)
diff
 tensor([[ 1.3046e-04,  2.6550e-05],
        [-1.3180e-04,  7.4476e-05],
        [ 1.5004e-04,  3.9122e-05]], device='cuda:0', dtype=torch.float64)
diffmean
 tensor(9.2075e-05, device='cuda:0', dtype=torch.float64)
